<a href="https://colab.research.google.com/github/TiagolGoulart/GeoPython/blob/main/downloadCBERS4A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Connecting with google drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# Installing the necessaries libraries

!pip install geopandas -q
!pip install rasterio -q
!pip install shapely -q
!pip install simplekml -q
!pip install cbers4asat[tools] -q

In [ ]:
!pip install numpy==1.23

In [4]:
from cbers4asat.tools import clip

In [5]:
# Importing packages

import geopandas as gpd
import simplekml
import numpy as np
import rasterio
import os
import lxml.etree as etree
import pyproj
from pyproj import Transformer
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.features import shapes
from shapely.geometry import Polygon, Point

In [ ]:
# Defining the data

# Define the coordinate system with EPSG code
sistema_coordenadas = input("Digite o código EPSG do sistema de coordenadas de referência: \n")
sistema_coordenadas = 'EPSG:'+sistema_coordenadas

# Path for mask to clip raster
kml_path = input('Cole o caminho para o kml que irá cortar as imagens raster: ')

# INPE user
user_cbers = input('Digite aqui seu e-mail de usuário cadastrado na plataforma do CBERS4A: \n')

# Inicial search date
inicial_date = input('Digite a data inicial de busca da imagem no formato DD/MM/AAAA: \n')
d_i = int(inicial_date[:2])
m_i = int(inicial_date[3:5])
a_i = int(inicial_date[6:])

# Final search date

final_date= input('Digite a data final de busca da imagem no formato DD/MM/AAAA: \n')
d_f = int(final_date[:2])
m_f = int(final_date[3:5])
a_f = int(final_date[6:])

# Setting the results output directory

out_dir = input('Cole o caminho da pasta onde os arquivos resultantes serão salvos: \n')

In [7]:
# Opening the extension KML and setting the bbox

# Open the KML file in binary mode
with open(kml_path, 'rb') as f:
  kml_string = f.read()

# parse the KML string into an Element object
root = etree.fromstring(kml_string)

# Get the coordinates of the polygon
polygon = root.find('.//{http://www.opengis.net/kml/2.2}Polygon')
coords_str = polygon.find('.//{http://www.opengis.net/kml/2.2}coordinates').text

coords_list_original = [tuple(map(float, coord.split(',')[0:2])) for coord in coords_str.split()]
coords_list = [(coord[1], coord[0]) for coord in coords_list_original]

# Create a Transformer for the proper UTM projection
transformer = Transformer.from_crs("EPSG:4326", sistema_coordenadas)

# Convert the coordinates to the proper UTM projection
coords_list_utm = [transformer.transform(*coord)[::-1] for coord in coords_list]
coords_list = [(coord[1], coord[0]) for coord in coords_list_utm]

# Create a shapely polygon based on UTM coordinates for cutting
polygon_utm = Polygon(coords_list)

# Defining the mask based on the initial polygon
geo = polygon_utm

# Create a shapely polygon with geographic coordinates to get bbox
polygon_bbox = Polygon(coords_list_original)

# Using the 'envelope' attribute to get a bounding rectangle and then getting the max and min values
envelope = polygon_bbox.envelope
x_min, y_min, x_max, y_max = envelope.bounds

In [8]:
# Importing the libraries
from cbers4asat import Cbers4aAPI

# The date class is necessary to standardize the date format
from datetime import date

# Instantiating the object with the user registered on the platform
api = Cbers4aAPI(user_cbers)

# Bounding box formed by [x_min, y_min, x_max, y_max]
bbox=[x_min,y_min,x_max,y_max]

# Search interval
data_inicial = date(a_i,m_i,d_i)
data_final = date(a_f,m_f,d_f)


# Querying the catalog and displaying the results
produtos = api.query(location=bbox,
                     initial_date=data_inicial,
                     end_date=data_final,
                     cloud=100,
                     limit=1,
                     collections=['CBERS4A_WPM_L4_DN'])


In [ ]:
# Viewing the data
produtos

In [10]:
# Downloading the images for the output directory

api.download(products=produtos,
             bands=['green','blue','red','nir'],
             threads=10,
             outdir=out_dir)



In [15]:
# Defining the path of each band image

import time
import glob
start_time = time.time()  # Get the current time before starting the loop
timeout = 60  # Timeout in seconds

while True:
    try:
        blue_path = glob.glob(out_dir + '/*BAND1.tif')[0]
        green_path = glob.glob(out_dir + '/*BAND2.tif')[0]
        red_path = glob.glob(out_dir + '/*BAND3.tif')[0]
        nir_path = glob.glob(out_dir + '/*BAND4.tif')[0]

        break  # Exit the loop if no error occurs
    except IndexError:
        if time.time() - start_time >= timeout:
            print("Timeout: The timeout limit of 60 seconds has been reached."
            + "Please wait a moment and run the cell again.")
            break  # Exit the loop if the timeout is reached

In [16]:
# Defining a reprojection function of the downloaded rasters

import rasterio
from rasterio.warp import calculate_default_transform, reproject

# input_file and output_file refer to input and output paths of the initial raster
# and the reprojected raster, respectively, since dst_crs is the desired reference system
# which should be typed in the pattern 'EPSG:4326' (Geographic WGS84 in this case)

def reproject_raster(input_file, output_file, dst_crs):
    # Opening the input raster
    with rasterio.open(input_file) as src:
        # Calculate the transformation and dimensions for the reprojected raster
        transform, width, height = calculate_default_transform(src.crs, dst_crs, src.width, src.height, *src.bounds)

        # Create the output raster file
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open(output_file, 'w', **kwargs) as dst:
            # Reproject input raster to output crs
            reproject(
                source=rasterio.band(src, 1),
                destination=rasterio.band(dst, 1),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=rasterio.enums.Resampling.nearest
            )

In [17]:
# Reprojecting the rasters with the defined function

blue_path_rprj = out_dir+'/RPRJ_BAND1.tif'
green_path_rprj = out_dir+'/RPRJ_BAND2.tif'
red_path_rprj = out_dir+'/RPRJ_BAND3.tif'
nir_path_rprj = out_dir+'/RPRJ_BAND4.tif'


reproject_raster(blue_path,
                 blue_path_rprj,
                 sistema_coordenadas)

reproject_raster(green_path,
                 green_path_rprj,
                 sistema_coordenadas)

reproject_raster(red_path,
                 red_path_rprj,
                 sistema_coordenadas)

reproject_raster(nir_path,
                 nir_path_rprj,
                 sistema_coordenadas)

In [18]:
# Cutting the rasters by the extension of the project

# Defining the mask based on the initial polygon
geo = polygon_utm

from cbers4asat.tools import clip

out_dir_rec = out_dir+"/Recortados"

clip(
    raster=blue_path_rprj,
    mask=geo,
    outdir=out_dir_rec,
    filename="BAND1_REC.tif",
)

clip(
    raster=green_path_rprj,
    mask=geo,
    outdir=out_dir_rec,
    filename="BAND2_REC.tif",
)

clip(
    raster=red_path_rprj,
    mask=geo,
    outdir=out_dir_rec,
    filename="BAND3_REC.tif",
)

clip(
    raster=nir_path_rprj,
    mask=geo,
    outdir=out_dir_rec,
    filename="BAND4_REC.tif",
)

In [19]:
# Saving the paths to the rasters in variables

blue_path=out_dir_rec+'/BAND1_REC.tif'
green_path=out_dir_rec+'/BAND2_REC.tif'
red_path=out_dir_rec+'/BAND3_REC.tif'
nir_path=out_dir_rec+'/BAND4_REC.tif'

In [ ]:
# Displaying the analysis image above

from cbers4asat.tools import rgbn_composite
import rasterio as rio
from rasterio.plot import show

# Creating a TRUE COLOR composite
# Each color parameter represents the output image channel
out_dir_RGB = '/content/drive/MyDrive/#FINAL_RESULT'

# NIR band is optional
rgbn_composite(red=red_path,
               green=green_path,
               blue=blue_path,
               nir=nir_path,
               filename='TRUE_COLOR.tif',
               outdir=out_dir_RGB)

raster = rio.open(out_dir_RGB + "/TRUE_COLOR.tif")

show(raster.read(), transform=raster.transform)

In [26]:
# Removing files to liberate memory

import os

# Home folder path
pasta = out_dir

# Get the list of files in the folder
arquivos = os.listdir(pasta)

# Go through the list of files and delete them
for arquivo in arquivos:
    caminho_arquivo = os.path.join(pasta, arquivo)
    if os.path.isfile(caminho_arquivo):
        os.remove(caminho_arquivo)


# Clipped folder path
pasta = out_dir_rec

# Get the list of files in the folder
arquivos = os.listdir(pasta)

# Go through the list of files and delete them
for arquivo in arquivos:
    caminho_arquivo = os.path.join(pasta, arquivo)
    if os.path.isfile(caminho_arquivo):
        os.remove(caminho_arquivo)